# Setup

In [57]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [77]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from generative_social_choice.utils.helper_functions import get_base_dir_path

In [59]:
from generative_social_choice.slates.voting_algorithms import (
    SequentialPhragmenMinimax,
    GreedyTotalUtilityMaximization,
    ExactTotalUtilityMaximization,
    LPTotalUtilityMaximization,
    VotingAlgorithm,
    GeometricTransformation,
)
from generative_social_choice.slates.voting_algorithm_axioms import (
    IndividualParetoAxiom,
    HappiestParetoAxiom,
    CoverageAxiom,
    MinimumAndTotalUtilityParetoAxiom,
    VotingAlgorithmAxiom,
    NonRadicalMinUtilityAxiom,
    NonRadicalTotalUtilityAxiom,
)
from generative_social_choice.test.utilities_for_testing import rated_vote_cases

# Load Data

In [60]:
voting_algorithm_evals_dir = get_base_dir_path() / "data" / "voting_algorithm_evals"
latest = True
if latest:
    file = sorted(voting_algorithm_evals_dir.glob("*.csv"))[-1]
else:
    file = voting_algorithm_evals_dir / "2025-01-20-180945.csv"
file

df = pd.read_csv(file, index_col=0, header=[0, 1])
df

vote                                                       Simple 1  \
subtest                                            Maximum Coverage   
GreedyTotalUtilityMaximization(utility_transfor...              1.0   
ExactTotalUtilityMaximization(utility_transform...              1.0   
LPTotalUtilityMaximization(utility_transform=None)              1.0   
GreedyTotalUtilityMaximization(utility_transfor...              1.0   
ExactTotalUtilityMaximization(utility_transform...              1.0   
LPTotalUtilityMaximization(utility_transform=Ge...              1.0   
GreedyTotalUtilityMaximization(utility_transfor...              1.0   
ExactTotalUtilityMaximization(utility_transform...              1.0   
LPTotalUtilityMaximization(utility_transform=Ge...              1.0   
Phragmen(marginal_slate, clear=True, redist=True)               1.0   
Phragmen(marginal_slate, clear=True, redist=False)              1.0   
Phragmen(marginal_slate, clear=False, redist=True)              1.0   
Phragmen(marginal_slate, clear=False, redist=Fa...              1.0   
Phragmen(marginal_previous, clear=True, redist=...              1.0   
Phragmen(marginal_previous, clear=True, redist=...              1.0   
Phragmen(marginal_previous, clear=False, redist...              1.0   
Phragmen(marginal_previous, clear=False, redist...              1.0   

vote                                                                                       \
subtest                                            m-th Happiest Person Pareto Efficiency   
GreedyTotalUtilityMaximization(utility_transfor...                                    1.0   
ExactTotalUtilityMaximization(utility_transform...                                    1.0   
LPTotalUtilityMaximization(utility_transform=None)                                    1.0   
GreedyTotalUtilityMaximization(utility_transfor...                                    1.0   
ExactTotalUtilityMaximization(utility_transform...                                    1.0   
LPTotalUtilityMaximization(utility_transform=Ge...                                    1.0   
GreedyTotalUtilityMaximization(utility_transfor...                                    1.0   
ExactTotalUtilityMaximization(utility_transform...                                    1.0   
LPTotalUtilityMaximization(utility_transform=Ge...                                    1.0   
Phragmen(marginal_slate, clear=True, redist=True)                                     1.0   
Phragmen(marginal_slate, clear=True, redist=False)                                    1.0   
Phragmen(marginal_slate, clear=False, redist=True)                                    1.0   
Phragmen(marginal_slate, clear=False, redist=Fa...                                    1.0   
Phragmen(marginal_previous, clear=True, redist=...                                    1.0   
Phragmen(marginal_previous, clear=True, redist=...                                    1.0   
Phragmen(marginal_previous, clear=False, redist...                                    1.0   
Phragmen(marginal_previous, clear=False, redist...                                    1.0   

vote                                                                             \
subtest                                            Individual Pareto Efficiency   
GreedyTotalUtilityMaximization(utility_transfor...                          1.0   
ExactTotalUtilityMaximization(utility_transform...                          1.0   
LPTotalUtilityMaximization(utility_transform=None)                          1.0   
GreedyTotalUtilityMaximization(utility_transfor...                          1.0   
ExactTotalUtilityMaximization(utility_transform...                          1.0   
LPTotalUtilityMaximization(utility_transform=Ge...                          1.0   
GreedyTotalUtilityMaximization(utility_transfor...                          1.0   
ExactTotalUtilityMaximization(utility_transform...                          1.0   
LPTotalUtilityMaximization(utility_transform=Ge...  

In [61]:
# Select subset of rows
# df = df.iloc[3:9]
df

vote                                                       Simple 1  \
subtest                                            Maximum Coverage   
GreedyTotalUtilityMaximization(utility_transfor...              1.0   
ExactTotalUtilityMaximization(utility_transform...              1.0   
LPTotalUtilityMaximization(utility_transform=None)              1.0   
GreedyTotalUtilityMaximization(utility_transfor...              1.0   
ExactTotalUtilityMaximization(utility_transform...              1.0   
LPTotalUtilityMaximization(utility_transform=Ge...              1.0   
GreedyTotalUtilityMaximization(utility_transfor...              1.0   
ExactTotalUtilityMaximization(utility_transform...              1.0   
LPTotalUtilityMaximization(utility_transform=Ge...              1.0   
Phragmen(marginal_slate, clear=True, redist=True)               1.0   
Phragmen(marginal_slate, clear=True, redist=False)              1.0   
Phragmen(marginal_slate, clear=False, redist=True)              1.0   
Phragmen(marginal_slate, clear=False, redist=Fa...              1.0   
Phragmen(marginal_previous, clear=True, redist=...              1.0   
Phragmen(marginal_previous, clear=True, redist=...              1.0   
Phragmen(marginal_previous, clear=False, redist...              1.0   
Phragmen(marginal_previous, clear=False, redist...              1.0   

vote                                                                                       \
subtest                                            m-th Happiest Person Pareto Efficiency   
GreedyTotalUtilityMaximization(utility_transfor...                                    1.0   
ExactTotalUtilityMaximization(utility_transform...                                    1.0   
LPTotalUtilityMaximization(utility_transform=None)                                    1.0   
GreedyTotalUtilityMaximization(utility_transfor...                                    1.0   
ExactTotalUtilityMaximization(utility_transform...                                    1.0   
LPTotalUtilityMaximization(utility_transform=Ge...                                    1.0   
GreedyTotalUtilityMaximization(utility_transfor...                                    1.0   
ExactTotalUtilityMaximization(utility_transform...                                    1.0   
LPTotalUtilityMaximization(utility_transform=Ge...                                    1.0   
Phragmen(marginal_slate, clear=True, redist=True)                                     1.0   
Phragmen(marginal_slate, clear=True, redist=False)                                    1.0   
Phragmen(marginal_slate, clear=False, redist=True)                                    1.0   
Phragmen(marginal_slate, clear=False, redist=Fa...                                    1.0   
Phragmen(marginal_previous, clear=True, redist=...                                    1.0   
Phragmen(marginal_previous, clear=True, redist=...                                    1.0   
Phragmen(marginal_previous, clear=False, redist...                                    1.0   
Phragmen(marginal_previous, clear=False, redist...                                    1.0   

vote                                                                             \
subtest                                            Individual Pareto Efficiency   
GreedyTotalUtilityMaximization(utility_transfor...                          1.0   
ExactTotalUtilityMaximization(utility_transform...                          1.0   
LPTotalUtilityMaximization(utility_transform=None)                          1.0   
GreedyTotalUtilityMaximization(utility_transfor...                          1.0   
ExactTotalUtilityMaximization(utility_transform...                          1.0   
LPTotalUtilityMaximization(utility_transform=Ge...                          1.0   
GreedyTotalUtilityMaximization(utility_transfor...                          1.0   
ExactTotalUtilityMaximization(utility_transform...                          1.0   
LPTotalUtilityMaximization(utility_transform=Ge...  

## Overall Performance

In [74]:
df.sum(axis=1)

GreedyTotalUtilityMaximization(utility_transform=None)                               144.333333
ExactTotalUtilityMaximization(utility_transform=None)                                157.250000
LPTotalUtilityMaximization(utility_transform=None)                                   154.083333
GreedyTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))     146.666667
ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))      161.333333
LPTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))         155.625000
GreedyTotalUtilityMaximization(utility_transform=GeometricTransformation(p=10.0))    143.666667
ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=10.0))     155.750000
LPTotalUtilityMaximization(utility_transform=GeometricTransformation(p=10.0))        155.458333
Phragmen(marginal_slate, clear=True, redist=True)                                    143.166667
Phragmen(marginal_slate, clear=True, red

In [80]:
pd.DataFrame(np.trunc(df.values), index=df.index, columns=df.columns).sum(axis=1)

GreedyTotalUtilityMaximization(utility_transform=None)                               130.0
ExactTotalUtilityMaximization(utility_transform=None)                                152.0
LPTotalUtilityMaximization(utility_transform=None)                                   152.0
GreedyTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))     135.0
ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))      154.0
LPTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))         154.0
GreedyTotalUtilityMaximization(utility_transform=GeometricTransformation(p=10.0))    129.0
ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=10.0))     148.0
LPTotalUtilityMaximization(utility_transform=GeometricTransformation(p=10.0))        154.0
Phragmen(marginal_slate, clear=True, redist=True)                                    126.0
Phragmen(marginal_slate, clear=True, redist=False)                                   127.0

## Differential Test Performance

In [63]:
non_uniform_columns = df.loc[:, df.nunique() > 1]
non_uniform_columns


vote                                                       Simple 3  \
subtest                                            Maximum Coverage   
GreedyTotalUtilityMaximization(utility_transfor...         1.000000   
ExactTotalUtilityMaximization(utility_transform...         1.000000   
LPTotalUtilityMaximization(utility_transform=None)         1.000000   
GreedyTotalUtilityMaximization(utility_transfor...         1.000000   
ExactTotalUtilityMaximization(utility_transform...         1.000000   
LPTotalUtilityMaximization(utility_transform=Ge...         1.000000   
GreedyTotalUtilityMaximization(utility_transfor...         1.000000   
ExactTotalUtilityMaximization(utility_transform...         1.000000   
LPTotalUtilityMaximization(utility_transform=Ge...         1.000000   
Phragmen(marginal_slate, clear=True, redist=True)          0.208333   
Phragmen(marginal_slate, clear=True, redist=False)         0.416667   
Phragmen(marginal_slate, clear=False, redist=True)         0.416667   
Phragmen(marginal_slate, clear=False, redist=Fa...         0.458333   
Phragmen(marginal_previous, clear=True, redist=...         0.583333   
Phragmen(marginal_previous, clear=True, redist=...         0.541667   
Phragmen(marginal_previous, clear=False, redist...         0.375000   
Phragmen(marginal_previous, clear=False, redist...         0.458333   

vote                                                                             \
subtest                                            Individual Pareto Efficiency   
GreedyTotalUtilityMaximization(utility_transfor...                     1.000000   
ExactTotalUtilityMaximization(utility_transform...                     1.000000   
LPTotalUtilityMaximization(utility_transform=None)                     1.000000   
GreedyTotalUtilityMaximization(utility_transfor...                     1.000000   
ExactTotalUtilityMaximization(utility_transform...                     1.000000   
LPTotalUtilityMaximization(utility_transform=Ge...                     1.000000   
GreedyTotalUtilityMaximization(utility_transfor...                     1.000000   
ExactTotalUtilityMaximization(utility_transform...                     1.000000   
LPTotalUtilityMaximization(utility_transform=Ge...                     1.000000   
Phragmen(marginal_slate, clear=True, redist=True)                      0.666667   
Phragmen(marginal_slate, clear=True, redist=False)                     0.625000   
Phragmen(marginal_slate, clear=False, redist=True)                     0.583333   
Phragmen(marginal_slate, clear=False, redist=Fa...                     0.833333   
Phragmen(marginal_previous, clear=True, redist=...                     0.750000   
Phragmen(marginal_previous, clear=True, redist=...                     0.625000   
Phragmen(marginal_previous, clear=False, redist...                     0.583333   
Phragmen(marginal_previous, clear=False, redist...                     0.666667   

vote                                                                                                    \
subtest                                            Minimum Utility and Total Utility Pareto Efficiency   
GreedyTotalUtilityMaximization(utility_transfor...                                           1.000000    
ExactTotalUtilityMaximization(utility_transform...                                           1.000000    
LPTotalUtilityMaximization(utility_transform=None)                                           1.000000    
GreedyTotalUtilityMaximization(utility_transfor...                                           1.000000    
ExactTotalUtilityMaximization(utility_transform...                                           1.000000    
LPTotalUtilityMaximization(utility_transform=Ge...                                           1.000000    
GreedyTotalUtilityMaximization(utility_transfor...                                           1.000000    
ExactTotalUtilityMaximization(utility_transform...                                           1.000000    
LPTotalUtili

These counts show which test cases and which axioms most frequently show unique behavior across the `VotingAlgorithm`s

In [ ]:
# Count the number of columns with each name in both levels of the MultiIndex
level_0_counts = non_uniform_columns.columns.get_level_values(0).value_counts()
level_1_counts = non_uniform_columns.columns.get_level_values(1).value_counts()

print("Counts for level 0:")
print(level_0_counts)



Counts for level 0:
vote
Simple 3           5
Ex A.1             5
Ex Alg1.5          5
Ex 1.3             5
Ex 3.1             5
Ex 4.2             5
Ex Alg1.3          5
Ex 1.1 modified    4
Ex 4.3             4
Ex D.1             4
Ex Alg A.2         4
Ex Alg A.1         4
Ex C.2             4
Ex 2.1             2
Ex B.3             2
Ex 4.1             1
Ex Alg1.4          1
Ex Alg2.1          1
Name: count, dtype: int64

Counts for level 1:
subtest
Non-radical Total Utility Pareto Efficiency            17
Maximum Coverage                                       14
Minimum Utility and Total Utility Pareto Efficiency    14
Non-radical Minimum Utility Pareto Efficiency          14
Individual Pareto Efficiency                            7
Name: count, dtype: int64


In [81]:
print("\nCounts for level 1:")
print(level_1_counts)


Counts for level 1:
subtest
Non-radical Total Utility Pareto Efficiency            17
Maximum Coverage                                       14
Minimum Utility and Total Utility Pareto Efficiency    14
Non-radical Minimum Utility Pareto Efficiency          14
Individual Pareto Efficiency                            7
Name: count, dtype: int64


# Debuggign

In [92]:
case = rated_vote_cases["Ex 3.1"]
alg = ExactTotalUtilityMaximization()
axiom = MinimumAndTotalUtilityParetoAxiom()
# axiom = NonRadicalTotalUtilityAxiom()

In [93]:
df1 = df.loc[str(alg),:].unstack()
df1

subtest,Individual Pareto Efficiency,Maximum Coverage,Minimum Utility and Total Utility Pareto Efficiency,Non-radical Minimum Utility Pareto Efficiency,Non-radical Total Utility Pareto Efficiency,m-th Happiest Person Pareto Efficiency
vote,,,,,,
Ex 1.1,1.000000,1.000,1.00,1.000000,1.000000,1.0
Ex 1.1 modified,1.000000,1.000,1.00,1.000000,0.083333,1.0
Ex 1.2,1.000000,1.000,1.00,1.000000,1.000000,1.0
Ex 1.3,1.000000,1.000,1.00,1.000000,1.000000,1.0
Ex 2.1,1.000000,1.000,1.00,1.000000,1.000000,1.0
Ex 2.2,1.000000,1.000,1.00,1.000000,1.000000,1.0
Ex 3.1,0.916667,0.875,0.75,0.916667,0.875000,1.0
Ex 4.1,1.000000,1.000,1.00,1.000000,0.250000,1.0
Ex 4.2,1.000000,1.000,1.00,1.000000,1.000000,1.0


In [94]:

case.rated_votes

,s1,s2,s3,s4,s5,s6
0,2,0,1,0,0,0
1,1,2,0,0,0,0
2,0,1,2,0,0,0
3,0,0,0,2,0,1
4,0,0,0,1,2,0
5,0,0,0,0,1,2


In [95]:
alg.vote(rated_votes=case.rated_votes, slate_size=case.slate_size)[0]

['s1', 's2', 's5']

In [96]:
axiom.satisfactory_slates(rated_votes=case.rated_votes, slate_size=case.slate_size)

{frozenset({'s1', 's3', 's4'}),
 frozenset({'s2', 's3', 's4'}),
 frozenset({'s3', 's4', 's6'}),
 frozenset({'s3', 's4', 's5'}),
 frozenset({'s2', 's5', 's6'}),
 frozenset({'s1', 's5', 's6'}),
 frozenset({'s2', 's4', 's6'}),
 frozenset({'s1', 's2', 's6'}),
 frozenset({'s2', 's3', 's5'}),
 frozenset({'s2', 's4', 's5'}),
 frozenset({'s1', 's3', 's6'}),
 frozenset({'s1', 's4', 's6'}),
 frozenset({'s1', 's3', 's5'}),
 frozenset({'s1', 's4', 's5'}),
 frozenset({'s1', 's2', 's4'}),
 frozenset({'s2', 's3', 's6'}),
 frozenset({'s1', 's2', 's5'}),
 frozenset({'s3', 's5', 's6'})}

In [97]:
num_aug_cases = 24
aug_case_votes = pd.DataFrame(index = range(num_aug_cases), columns=["rated_votes", "assignments", "axiom_slates", "alg_slate", "satisfied"])
for i, aug_case in enumerate(case.augmented_cases[:num_aug_cases]):
    # print(aug_case)
    aug_case_votes.at[i, "rated_votes"] = aug_case
    aug_case_votes.at[i, "axiom_slates"] = axiom.satisfactory_slates(rated_votes=aug_case, slate_size=case.slate_size)
    aug_case_votes.at[i, "alg_slate"], aug_case_votes.at[i, "assignments"] = alg.vote(rated_votes=aug_case, slate_size=case.slate_size)
    aug_case_votes.at[i, "satisfied"] = axiom.evaluate_assignment(
        rated_votes=aug_case,
        assignments=aug_case_votes.at[i, "assignments"],
        slate_size=case.slate_size,
    )
    # print(axiom.satisfactory_slates(rated_votes=aug_case, slate_size=case.slate_size))
aug_case_votes.iloc[:,2:]

,axiom_slates,alg_slate,satisfied
0,"{(s1, s2, s6), (s1, s3, s5)}","[s1, s2, s6]",True
1,"{(s3, s5, s2)}","[s2, s3, s5]",True
2,"{(s4, s3, s2), (s4, s3, s5), (s1, s2, s6), (s3...","[s1, s2, s5]",True
3,"{(s4, s1, s3)}","[s1, s3, s4]",True
4,"{(s1, s5, s6)}","[s1, s5, s6]",True
5,"{(s4, s5, s2)}","[s2, s4, s5]",True
6,"{(s1, s3, s5)}","[s1, s3, s5]",True
7,"{(s4, s5, s2)}","[s2, s4, s5]",True
8,"{(s4, s3, s5)}","[s3, s4, s5]",True
9,"{(s4, s1, s2)}","[s1, s2, s4]",True


In [98]:
failures = aug_case_votes[aug_case_votes["satisfied"] == False]
failures.iloc[:,2:]


,axiom_slates,alg_slate,satisfied


In [99]:
f0 = failures.iloc[0]
f0.rated_votes
# (f0.rated_votes - case.rated_votes)*1e6


IndexError: single positional indexer is out-of-bounds

In [ ]:
((f0.rated_votes - case.rated_votes)*1e6)

,s1,s2,s3,s4
0,-36.913405,-37.436229,-35.498165,-37.443308
1,-45.803543,-44.253616,-44.329617,-45.232909
2,-93.509296,-98.007234,-93.420172,-94.746571
3,-85.813903,-85.000996,-82.870183,-84.180419
4,-26.317317,-25.807794,-25.528477,-27.988286
5,-51.628286,-53.502983,-54.687441,-51.042631


In [ ]:
f0.assignments

,candidate_id
0,s1
1,s3
2,s1
3,s3
4,s3
5,s1


In [ ]:
alg.vote(rated_votes=f0.rated_votes, slate_size=case.slate_size)

(['s1', 's3'],
   candidate_id
 0           s1
 1           s3
 2           s1
 3           s3
 4           s3
 5           s1)

In [ ]:
axiom.evaluate_assignment(
    rated_votes=f0.rated_votes,
    assignments=f0.assignments,
    slate_size=case.slate_size,
)

False